# Linear Regression
In linear regression our predictors are linear functions $f : \R^d\to\R$ each parameterized by a vector $\bw\in\R^d$ of real coefficients. That is, $f(\bx) = \bw^{\top}\bx$.

Given a training set $(\bx_1,y_1),\ldots,(\bx_m,y_m) \in \R^d\times\R$, the linear regression predictor is the Empirical Risk Minimizer with respect to the square loss,
$$
    \bwhat = \argmin_{\bw\in\R^d} \sum_{t=1}^m \big(\bw^{\top}\bx_t - y_t\big)^2
$$
Now let $\bv = \big(\bw^{\top}\bx_1,\ldots,\bw^{\top}\bx_m\big)$ and $\by = (y_1,\ldots,y_m)$. Then
$$
     \sum_{t=1}^m \big(\bw^{\top}\bx_t - y_t\big)^2 = \norm{\bv-\by}^2.
$$
Since $\bv = S\bw$, where $S$ is a $m \times d$ matrix such that $S^{\top} = [\bx_1,\ldots,\bx_m]$, we may also write
$$
    \bwhat = \argmin_{\bw\in\R^d} \norm{S\bw-\by}^2
$$
Since $F(\bw) = \norm{S\bw-\by}^2$ is a convex function, the minimizer satisfies the condition $\nabla F(\bw) = \bzero$.

Using matrix calculus, we have that $\nabla\norm{S\bw-\by}^2 = 2S^{\top}(S\bw-\by)$. Hence, $\nabla\norm{S\bw-\by}^2 = \bzero$ for $\bw = \big(S^{\top}S\big)^{-1}S^{\top}\by$ provided $S^{\top}S$ is invertible.

Therefore, whenever $S^{\top}S$ is invertible we have that $\bwhat = \big(S^{\top}S\big)^{-1}S^{\top}\by$.

Since the Bayes optimal predictor for square loss is $f(\bx) = \E[Y \mid \bx]$, the Bayes optimal predictor is a linear function when $y = \bu^{\top}\bx + \ve_{\bx}$, where $\ve_{\bx}$ is an independent random variable such that $\E[\ve_{\bx} \mid \bx] = 0$.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn import decomposition
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score, validation_curve, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

#from auxCode import *
%matplotlib inline

We start by using the function `make_regression()` to generate a random dataset on which the Bayes optimal predictor is a linear function. We set $d=1$, $m=1000$, and $\sqrt{\Var[\ve_{\bx} \mid \bx]} = 10$.

In [ ]:
X, y, w = make_regression(n_samples=1000, n_features=1, n_informative=1, n_targets=1,
                          noise=10.0, shuffle=True, coef=True, random_state=42)

We can inspect the coefficient of the Bayes optimal predictor.

In [ ]:
np.round(w.item(), decimals=2)

The Bayes risk is the squared conditional variance, averaged over data points. In our case, Bayes risk $(10)^2 = 100$.

If we compute it on our dataset we get something close but not quite the same, which is reasonable considering we have sampled just $1000$ data points.

In [ ]:
y_opt = w.item()*X
np.round(mean_squared_error(y, y_opt), decimals=2)

Next, we split the dataset in train and test, with proportions 60% and 40%.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

We run a linear regression learner on the training data and print the learned coefficient.

In [ ]:
learner = LinearRegression()
learner.fit(X_train, y_train)
np.round(learner.coef_[0], decimals=2)

Note that the learned linear regressor is pretty close to the coefficient $16.75$ of the Bayes optimal predictor.

We then compute test predictions and measure test error with respect to the square loss.

In [ ]:
y_pred = learner.predict(X_test)
np.round(mean_squared_error(y_test, y_pred), decimals=2)

If we plot the test points and the two linear regressors (the one we learned and the Bayes optimal one) we can get an idea of how good is linear regression when the Bayes optimal predictor is also a linear function.

In [ ]:
x_min = np.min(X)
x_max = np.max(X)
v = np.arange(x_min, x_max, 0.2)

plt.scatter(X_test, y_test,  color='yellow')
plt.plot(X_test, y_pred, color='blue', linewidth=2)
plt.plot(v, v*w.item(), color='red', linewidth=2)
plt.show()

The test error of the Bayes optimal predictor is only slightly lower than that of the linear regressor learned on the training set.

In [ ]:
y_opt = w.item()*X_test
np.round(mean_squared_error(y_test, y_opt), decimals=2)

Now we keep the same training set and change the labels to make the Bayes optimal predictor a nonlinear function of $x$, namely $y = \sin(x) + 0.3\ve_x$, where $\ve_x$ is an independent Gaussian random variable with zero mean and variance equal to $0.25$.

In [ ]:
y = X * np.sin(X) + 0.5*np.random.randn(1000,1)

In [ ]:
plt.scatter(X, y,  color='yellow')
plt.show()

If we train on the entire dataset, linear regression achieves a training error of $0.58$.

In [ ]:
learner.fit(X, y)
y_pred = learner.predict(X)
np.round(mean_squared_error(y, y_pred), decimals=2)

We now use a trick to make linear regression more powerful at the expense of increasing the number of features.

The function `PolynomialFeatures()` replaces the original features $\bx = (x_1,\ldots,x_d)$ with all polynomial combinations of features with degree less than or equal to a specified degree $n$. For instance, if $d=2$ and $n=2$, the original features $(x_1,x_2)$ are replaced by $(1,x_1,x_2,x_1x_2,x_1^2,x_2^2)$. Note that the number of new features is of order $d^n$.

With $n=5$ the training error of linear regression goes down a lot.

In [ ]:
poly = PolynomialFeatures(degree=5, interaction_only=False, include_bias=True)
X_poly = poly.fit_transform(X)
learner.fit(X_poly, y)
y_pred = learner.predict(X_poly)

In [ ]:
np.round(mean_squared_error(y, y_pred), decimals=2)

In [ ]:
plt.scatter(X, y,  color='yellow')
plt.scatter(X, y_pred,  color='red')
plt.show()

We are now ready to try out linear regression on a real-world dataset.

The Boston Housing Dataset originates from the UCI Machine Learning Repository. The data was collected in $1978$ and each of the $506$ entries represent aggregated data about $14$ features for homes from various suburbs in Boston, Massachusetts. The label is the column `MEDV`.

We load the dataset and add column names. Note the the field separator in the file is whitespace instead of comma.

In [ ]:
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
housing = pd.read_csv("Datasets/housing.csv", delim_whitespace=True, names=names)
housing.info()

In [ ]:
housing.describe()

In [ ]:
housing.head()

We take a look at the distribution of the labels using the `distplot()` function of Seaborn.

In [ ]:
sns.distplot(housing['MEDV']);
fig = plt.figure()

We also look at the correlation matrix between features. The DataFrame method `corr()` of Pandas computes the Pearson correlation coefficient of the sample, defined for each pair $i,j$ of features by
$$
    r_{i,j} = \frac{\sum_{t=1}^m (x_{i,t}-\mu_i)(x_{j,t}-\mu_j)}{\sqrt{\sum_{t=1}^m (x_{i,t}-\mu_i)^2}\sqrt{\sum_{t=1}^m (x_{j,t}-\mu_j)^2}}
$$
where $\mu_i = (x_{i,1}+\cdots+x_{i,m})/m$.

Recall that the Pearson correlation coefficient $r_{i,j}$ is always in the $[-1,+1]$ interval. Values of $r_{i,j}$ closer to the extremes indicate that $i$ and $j$ tend to be linearly correlated, whereas values of $r_{i,j}$ close to zero indicate that $i$ and $j$ are nearly independent.

In [ ]:
corr=housing.corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corr, vmax=.8, linewidths=0.01,
            square=True,annot=True,cmap='YlGnBu',linecolor="white")
plt.title('Correlation between features');

Pairs of features with correlation coefficient close to $-1$ or $+1$ are redundant (for example, `TAX` and `RAD` in this dataset), and we could only keep one for each pair.

We check which feature has a correlation larger than $0.75$ in absolute value with at least some other feature.

In [ ]:
corr[((corr > 0.75) | (corr < -0.75)) & (corr != 1.0)]

It turns out that we can safely drop `NOX` and `RAD`.

In [ ]:
X = housing.drop(columns=['NOX', 'RAD', 'MEDV']).values
y = housing['MEDV'].values

We now set the linear regression learner with the (negative) square loss as scoring function. In our experiments, use the setting `normalize=True` which applies the data transformation $\bx_t \to (\bx_t - \bmu)/\norm{\bx_t}$ where $\bmu$ is the average of the data points.

In [ ]:
learner = LinearRegression(normalize=True)
RMS = 'neg_mean_squared_error'

The cross-validated risk estimate is $34.77$

In [ ]:
scores = cross_val_score(learner, X, y, cv=5, scoring=RMS)
np.round(-scores.mean(), decimals=2)

We now look at the learning curve for training set values from $150$ to $300$

In [ ]:
sizes = range(150, 301, 25)
train_size, train_score, val_score = learning_curve(learner, X, y, train_sizes=sizes, cv=5, scoring=RMS)

In [ ]:
train_score_mean = -np.mean(train_score, axis=1)
train_score_std = np.std(train_score, axis=1)
val_score_mean = -np.mean(val_score, axis=1)
val_score_std = np.std(val_score, axis=1)
plt.grid()
plt.fill_between(sizes, train_score_mean - train_score_std,
                 train_score_mean + train_score_std, alpha=0.1,
                 color="r")
plt.fill_between(sizes, val_score_mean - val_score_std,
                 val_score_mean + val_score_std, alpha=0.1, color="g")
plt.plot(sizes, train_score_mean, 'o-', color="r",
         label="Training error")
plt.plot(sizes, val_score_mean, 'o-', color="g",
         label="CV error")
plt.legend()
plt.xlabel('Training size')
plt.ylabel('Square loss')
plt.show()

Linear regression look **unstable** here. The risk estimates oscillate between $40$ and $80$ and have a huge standard deviation. 

We repeat the experiment without cross-validation so that we can extract the linear model $\bwhat$ learned by linear regression for each training set size

In [ ]:
coef_list = []

for s in range(150, 301, 25):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=s, random_state=42)
    learner.fit(X_train, y_train)
    coef_list.append(learner.coef_)

coef_matrix = np.array(coef_list)

Then, we use PCA to visually check how different the learned models are.

In [ ]:
pca = decomposition.PCA(n_components=2)
pca.fit(coef_matrix)
coef_pca = pca.transform(coef_matrix)

fig, ax = plt.subplots()
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-0.5, 0.5)
plt.scatter(coef_pca[:,0], coef_pca[:,1])

In order to increase the stability of linear regression, we first use PCA to reduce the number of features and increase the bias.

In [ ]:
pca = decomposition.PCA(n_components=11)
pca.fit(X)

plt.title('PCA')
plt.plot(pca.singular_values_, label='Singular values')
plt.legend()
plt.show()

We project all the datapoints on the six principal components. Then we check the learning curve again.

In [ ]:
pca = decomposition.PCA(n_components=6)
pca.fit(X)
X_pca = pca.transform(X)

In [ ]:
sizes = range(150, 301, 25)
train_size, train_score, val_score = learning_curve(learner, X_pca, y, train_sizes=sizes, cv=5, scoring=RMS)

In [ ]:
plt.title('PCA Linear Regression')
train_score_mean = -np.mean(train_score, axis=1)
train_score_std = np.std(train_score, axis=1)
val_score_mean = -np.mean(val_score, axis=1)
val_score_std = np.std(val_score, axis=1)
plt.grid()
plt.fill_between(sizes, train_score_mean - train_score_std,
                 train_score_mean + train_score_std, alpha=0.1,
                 color="r")
plt.fill_between(sizes, val_score_mean - val_score_std,
                 val_score_mean + val_score_std, alpha=0.1, color="g")
plt.plot(sizes, train_score_mean, 'o-', color="r",
         label="Training error")
plt.plot(sizes, val_score_mean, 'o-', color="g",
         label="CV risk estimate")
plt.legend()
plt.xlabel('Training size')
plt.ylabel('Square loss')
plt.show()

The learning curve looks more stable now. We doublecheck by repeating the procedure to plot the spread of the vectors learned using increasing sizes of the training set.

In [ ]:
coef_list = []

for s in range(150, 301, 25):
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y, train_size=s, random_state=42)
    learner.fit(X_train, y_train)
    coef_list.append(learner.coef_)
coef_matrix = np.array(coef_list)

pca = decomposition.PCA(n_components=2)
pca.fit(coef_matrix)
coef_pca = pca.transform(coef_matrix)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-0.5, 0.5)
plt.scatter(coef_pca[:,0], coef_pca[:,1])

As the plot shows, the learned models are a indeed more stable after PCA.

## Ridge Regression

A better way to increase the bias in linear regression, and thus make the model more stable, is by introducing a regularizer in the ERM functional.

In other words, instead defining $\bwhat$ by
$$
    \bwhat = \argmin_{\bw\in\R^d} \norm{S\bw-\by}^2
$$
we use the regularized form, also known as *Ridge Regression*,
$$
    \bwhat = \argmin_{\bw\in\R^d} \norm{S\bw-\by}^2 + \alpha\norm{\bw}^2
$$
where $\alpha \ge 0$ is the regularization parameter. When $\alpha$ is zero we recover the standard linear regression solution. When $\alpha$ is very large, the solution $\bwhat$ becomes the zero vector. Hence $\alpha$ can be used to control the bias of the algorithm.

Similarly to before, we have that
$$
    \nabla\Big(\norm{S\bw-\by}^2 + \alpha\norm{\bw}^2\Big) = 2S^{\top}(S\bw-\by) + 2\alpha\bw
$$    
Hence, the gradient vanishes for $\bw = \big(\alpha I + S^{\top}S\big)^{-1}S^{\top}\by$. Note that we do not have to worry anymore about the invertibility of $S^{\top}S$. Indeed, if $\lambda_1 \ge\cdots\ge \lambda_d \ge 0$ are the eigenvalues of $S^{\top}S$, the eigenvalues of $\alpha I + S^{\top}S$ are simply
$
    \alpha+\lambda_1 \ge\cdots\ge \alpha+\lambda_d > 0
$. Hence, $\alpha I + S^{\top}S$ is invertible whenever $\alpha > 0$.

We start by investigating the sensitivity of the CV risk estimate to the choice of $\alpha$.

In [ ]:
alpha_vals = np.linspace(0.01,2.0,20)
train_score, val_score = validation_curve(Ridge(normalize=True), X, y, 'alpha', alpha_vals, cv=5, scoring=RMS)

In [ ]:
plt.title('Ridge Regression')
train_score_mean = -np.mean(train_score, axis=1)
train_score_std = np.std(train_score, axis=1)
val_score_mean = -np.mean(val_score, axis=1)
val_score_std = np.std(val_score, axis=1)
plt.grid()
plt.fill_between(alpha_vals, train_score_mean - train_score_std,
                 train_score_mean + train_score_std, alpha=0.1,
                 color="r")
plt.fill_between(alpha_vals, val_score_mean - val_score_std,
                 val_score_mean + val_score_std, alpha=0.1, color="g")
plt.plot(alpha_vals, train_score_mean, color="r",
         label="Training error")
plt.plot(alpha_vals, val_score_mean, color="g",
         label="CV risk estimate")
plt.legend()
plt.xlabel('Alpha')
plt.ylabel('Square loss')
plt.show()

We see that overfitting essentially disappears for $\alpha > 0.5$, and bias progressively increases. The variance of the CV risk estimate remains high though.

With normalized data (as performed by the flag `normalize=True`), the choice $\alpha = 1$ is often reasonable.

We now compare the learning curves of PCA Linear Regression and Ridge Regression.

In [ ]:
lin_reg = LinearRegression(normalize=True)

sizes = range(150, 301, 25)
l_train_size, l_train_score, l_val_score = learning_curve(lin_reg,
                                                          X_pca, y, train_sizes=sizes, cv=5, scoring=RMS)
r_train_size, r_train_score, r_val_score = learning_curve(Ridge(normalize=True),
                                                          X_pca, y, train_sizes=sizes, cv=5, scoring=RMS)

In [ ]:
plt.title('PCA Linear Regression vs. Ridge Regression')
l_val_score_mean = -np.mean(l_val_score, axis=1)
l_val_score_std = np.std(l_val_score, axis=1)
r_val_score_mean = -np.mean(r_val_score, axis=1)
r_val_score_std = np.std(r_val_score, axis=1)
plt.grid()
plt.fill_between(sizes, l_val_score_mean - l_val_score_std,
                 l_val_score_mean + l_val_score_std, alpha=0.1,
                 color="r")
plt.fill_between(sizes, r_val_score_mean - r_val_score_std,
                 r_val_score_mean + r_val_score_std, alpha=0.1, color="g")
plt.plot(sizes, l_val_score_mean, 'o-', color="r",
         label="PCA Linear Regression")
plt.plot(sizes, r_val_score_mean, 'o-', color="g",
         label="Ridge Regression")
plt.legend()
plt.xlabel('Training size')
plt.ylabel('CV risk estimate')
plt.show()

We see that, despite a large variance in the CV risk estimate, Ridge Regression is better than PCA Linear Regression, especially in the small training set regime.